<a href="https://colab.research.google.com/github/chenxu0602/TensorFlow2.0/blob/master/myLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

In [ ]:
!gsutil ls gs://mnist-4/data
!gsutil cp -r gs://mnist-4/data .

In [ ]:
def read_tfrecord(serialized_example):
  features = tf.io.parse_single_example(
      serialized_example,
      features={
          'height': tf.io.FixedLenFeature([], tf.int64),
          'width': tf.io.FixedLenFeature([], tf.int64),
          'depth': tf.io.FixedLenFeature([], tf.int64),
          'label': tf.io.FixedLenFeature([], tf.int64),
          'image_raw': tf.io.FixedLenFeature([], tf.string),
      })

  image = tf.io.decode_raw(features["image_raw"], tf.uint8)
  image_shape = [features["height"], features["width"], features["depth"]]
  image = tf.reshape(image, image_shape)
  image = tf.cast(image, tf.float32) * (1. / 255)
  label = tf.cast(features['label'], tf.int32)

  return image, label

In [ ]:
def input_fn_train(filename, batch_size=64):
  print(f"train data dir: {filename}")    
  dataset = tf.data.TFRecordDataset([filename])
  dataset = dataset.map(read_tfrecord)
  dataset = dataset.shuffle(60000).batch(batch_size) 
  return dataset

def input_fn_eval(filename, batch_size=64):
  print(f"eval data dir: {filename}")    
  dataset = tf.data.TFRecordDataset([filename])
  dataset = dataset.map(read_tfrecord)
  dataset = dataset.batch(batch_size) 
  return dataset

In [ ]:
def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
  ])

  model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
  print(model.summary())
  return model

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
with strategy.scope():
  model = create_model()

In [ ]:
earlyStopping_callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [ ]:
model.fit(input_fn_train("data/train.tfrecords"), 
          validation_data=input_fn_eval("data/test.tfrecords"), 
          epochs=100, 
          verbose=1,
          callbacks=[tensorboard_callback])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./logs